# Creando un Modelo Personalizado para NER

* Las entidades no pueden superponerse, por lo que cada token solo puede ser parte de una entidad.
* También es muy importante que el modelo aprenda palabras que *no* son entidades. En este caso las anotaciones serán vacías.
* El objetivo es enseñar al modelo a reconocer nuevas labels en contextos similares, incluso si no estaban en los datos de entrenamiento.

## Generando Datos de Entrenamiento

In [1]:
import spacy

nlp = spacy.load("pt_core_news_lg", 
                 disable = ["parser", "attribute_ruler", "lemmatizer", "ner"])

print(nlp.pipe_names)
ruler = nlp.add_pipe("entity_ruler")

un = ["km", "au"]
planets = ["mercurio", "venus", "tierra", "marte", "júpiter", "saturno", "urano", "neptuno"]

patterns = [{"label": "Intervalo",
             "pattern": [{"POS": "NUM"}, 
                         {"OP": "?", "TEXT": {"IN": un}}, 
                         {"LOWER": {"IN": ["a", "hasta", "-"]}}, 
                         {"POS": "NUM"}, 
                         {"TEXT": {"IN": un}}     
                         ]
            },
            {"label": "Magnitud", 
             "pattern": [{"POS": "NUM"},
                         {"TEXT": {"IN": un}}
                         ]
            },
            {"label": "Planeta",
             "pattern": [{"TEXT": {"IN": planets}}]
            },
            {
            "label": "Planeta",
            "pattern": [{"TEXT": "planeta"},
                        {"TEXT": {"IN": ["9", "nueve"]}}
                        ]             
            }
            ]
    
ruler.add_patterns(patterns)
print(nlp.pipe_names, end = "\n\n")

TEXTS = ["El Planeta 9 es un planeta hipotético en el sistema solar externo. ",
         "Trujillo y Shepherd analizaron las órbitas de TNOs con perihelio superior a 30 au y un semieje mayor "
         "más grande que 150 au",
         "El semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ",
         'Brown ha declarado: "En realidad lo llamamos “Fattie” (Gordito) cuando estamos hablando entre nosotros". ',
         "La capacidad para detectar el Planeta Nueve dependería de su ubicación y características. ",
         "La existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos "
         "transneptunianos."
        ]

TEXTS = [i.lower() for i in TEXTS]
    
TRAINING_DATA = []

for doc in nlp.pipe(TEXTS):
    spans = []
    for ent in doc.ents:
        span = {"start": ent.start_char, "end": ent.end_char, "label": ent.label}
        spans.append(span)    
    entities = [(span["start"], span["end"], doc.vocab.strings[span["label"]]) for span in spans]     
    training_example = (doc.text, {"entities": entities})
    TRAINING_DATA.append(training_example)       
    
print(TRAINING_DATA)    

['tok2vec', 'morphologizer']
['tok2vec', 'morphologizer', 'entity_ruler']

[('el planeta 9 es un planeta hipotético en el sistema solar externo. ', {'entities': [(3, 12, 'Planeta')]}), ('trujillo y shepherd analizaron las órbitas de tnos con perihelio superior a 30 au y un semieje mayor más grande que 150 au', {'entities': [(76, 81, 'Magnitud'), (116, 122, 'Magnitud')]}), ('el semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ', {'entities': [(52, 65, 'Intervalo')]}), ('brown ha declarado: "en realidad lo llamamos “fattie” (gordito) cuando estamos hablando entre nosotros". ', {'entities': []}), ('la capacidad para detectar el planeta nueve dependería de su ubicación y características. ', {'entities': [(30, 43, 'Planeta')]}), ('la existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos transneptunianos.', {'entities': []})]


## Entrenamiento

In [2]:
import random
from spacy.training import Example

labels = ["Intervalo", "Magnitud", "Planeta"]

nlp = spacy.blank("es")
print(nlp.pipe_names)

ner = nlp.add_pipe("ner")
print(nlp.pipe_names)

for ent in labels:
    ner.add_label(ent)

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):
    
    # Iniciar el entrenamiento    
    optimizer = nlp.begin_training()

    # Ciclo de 10 iteraciones
    for itn in range(10):
        # Barajar los dados de entrenamiento
        random.shuffle(TRAINING_DATA)
        losses = {}
        
        for text, annotation in TRAINING_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], sgd = optimizer, losses = losses)

        print(losses)

[]
['ner']
{'ner': 88.2028016448021}
{'ner': 31.50350563475513}
{'ner': 8.99412269949535}
{'ner': 13.980763114610898}
{'ner': 6.482054722926021}
{'ner': 25.618301453932347}
{'ner': 10.68182820707882}
{'ner': 5.5007339502598285}
{'ner': 3.2760314582196006}
{'ner': 2.967404100743649}


In [3]:
doc = nlp("Podría haber un pequeño planeta entre 60 - 100 au (además del Planeta 9).".lower())
[(ent.text, ent.label_) for ent in doc.ents]

[('60 - 100 au', 'Intervalo'), ('planeta 9', 'Planeta')]